In [1]:
%load_ext sql

In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import peakutils
import locate
import farwave
import time

from scipy import signal

MIC_IND_LR = {
    0: (1, 2),
    1: (2, 0),
    2: (0, 1)
}

ANGLE_OFFSET = {
    0: 0,
    1: 2.0944, # 120 deg in rad
    2: 4.18879, # 240 deg in rad
}


In [3]:
%matplotlib notebook

In [4]:
%sql sqlite:///schema/sunday_cardboard.db

u'Connected: None@schema/sunday_cardboard.db'

In [5]:
l = 0.3 # Length from mic to centre of array

def calc_delays(sigs, offsets):
    """ Calculate matrix of delays given N sigs
    """
    results = []
    for j in range(len(sigs)):
        for k in range(len(sigs)):
            if j != k:
                results.append((
                        (j, k), locate.xcorr_peaks(sigs[j], sigs[k], offsets[j], offsets[k], l)
                    ))
            

    # 3x3 array delays[i][j] is the delay of signal j relative to signal i
    delays = np.zeros((len(sigs), len(sigs)), dtype=np.float)
    for res in results:
        key, val = res[0], res[1]
        delays[key[0]][key[1]] = val[1] # xcorr (val, delay) tuple
        delays[key[1]][key[0]] = -delays[key[0]][key[1]]
        
    return delays

def calc_angles(sigs, delays, l):
    """ Given delays matrix, calculate angle using both farwave and solver
    """
    # Estimate "location" of sound source, create array record
    f_theta = np.deg2rad(farwave.calc_angle(delays, l))
    for j in range(len(sigs)):
        theta = 0
        if delays[j][(j+1)%3] >= 0 and delays[j][(j+2)%3] >= 0:
            lr = MIC_IND_LR[j]
            print 'Using microphone %d as closest mic - (%d left, %d right)' % (j, lr[0], lr[1])
            r, theta = locate.locate(delays[j][lr[0]], delays[j][lr[1]], l, r0=5., theta0=f_theta-ANGLE_OFFSET[j])
            theta += ANGLE_OFFSET[j]
            print 'Solver: r = %f, theta = %f' % (r, np.rad2deg(theta))
            break
    
    return np.rad2deg(theta), np.rad2deg(f_theta)


In [6]:
experiments = %sql select id, comment from experiment

Done.


In [7]:
ang_solver_raw = []
ang_solver_butter = []
ang_far_raw = []
ang_far_butter = []

for i in range(len(experiments)):
    cur_experiment_id = int(experiments[i][0])
    result = %sql select data from mic where experiment_id = :cur_experiment_id
        
    mic_0 = np.array(result[0][0])
    mic_1 = np.array(result[1][0])
    mic_2 = np.array(result[2][0])
    
    bufs = [mic_0, mic_1, mic_2]
    #try:
    try:
        sigs, sigs_filt = [], []
        for buf_i in bufs:
            sig, sig_filt = locate.filter_sigs(buf_i)
            sigs.append(sig)
            sigs_filt.append(sig_filt)
                
        sigs_cropped, sigs_butter_cropped, offsets, _, _ = locate.crop_sigs_npeaks(sigs, sigs_filt)
    except RuntimeError as e:
        print 'Error: %s' % str(e)
        solver_raw = solver_butter = far_raw = far_butter = 0
        ang_solver_raw.append(solver_raw)
        ang_solver_butter.append(solver_butter)
        ang_far_raw.append(far_raw)
        ang_far_butter.append(far_butter)
        continue
        
    # Calculate delays, angle using raw signal (cropped)
    delays_raw = calc_delays(sigs_cropped, offsets)
    solver_raw, far_raw = calc_angles(sigs_cropped, delays_raw, l)

    delays_butter = calc_delays(sigs_butter_cropped, offsets)
    solver_butter, far_butter = calc_angles(sigs_butter_cropped, delays_butter, l)
    #except:
    #    print 'Failure to find peaks: Setting to 0'
    #    solver_raw = far_raw = solver_butter = far_butter = 0
        
    print 'ID: %d, %f,%f,%f,%f' % (cur_experiment_id, far_raw, far_butter, solver_raw, solver_butter)
    
    ang_solver_raw.append(solver_raw)
    ang_solver_butter.append(solver_butter)
    ang_far_raw.append(far_raw)
    ang_far_butter.append(far_butter)


Done.
Angles: [-11.707039944049448, 12.664933465373002, 0.6551090708639373]
Errors: [1.5030314132153986, 1.4755579237395815, 1.0000653695691588]

Angle (CW): 0.644050
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 0.662633, theta = 419.999996


/usr/local/lib/python2.7/dist-packages/pandas/core/window.py:222: ComplexWarning: Casting complex values to real discards the imaginary part
  values = _ensure_float64(values)


Angles: [-10.899881992708337, 11.67737385323591, 0.5090863901257868]
Errors: [1.527325556529274, 1.5039053682124877, 1.0000394749873351]

Angle (CW): 0.510712
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 31.896839, theta = 422.600362
ID: 1, 359.355950,359.489288,419.999996,422.600362
Done.


/usr/local/lib/python2.7/dist-packages/scipy/optimize/minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


Angles: [-21.641451949378286, 21.977863134272745, 0.2683530666402641]
Errors: [1.2752781883033677, 1.2694014893698513, 1.0000109683634857]

Angle (CW): 0.247531
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 0.350861, theta = 419.999996
Angles: [-24.150365974082952, 24.325446008989413, 0.14206889673665574]
Errors: [1.2337198646719503, 1.2310076945241957, 1.0000030741415993]

Angle (CW): 0.129850
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 5.205827, theta = 426.486911
ID: 2, 359.752469,359.870150,419.999996,426.486911
Done.
Angles: [-22.282810028249056, 14.682362120308301, -5.692108599148913]
Errors: [1.264158534444168, 1.4221193591103174, 1.0049551929431084]

Angle (CW): -5.422860
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 0.339395, theta = 59.999997
Angles: [-23.862589547025966, 17.42858393415507, -4.974707958390545]
Errors: [1.2382290779746605, 1.357893686923101, 1.0037811692456446]

Angle (CW): -4.666675
Using microphon

/usr/local/lib/python2.7/dist-packages/scipy/optimize/minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


Angles: [93.52731400362812, 90.60183467105972, 107.23150560377606]
Errors: [1.1171349871997194, 1.1618102830981163, 3.3757178285877836]

Angle (CW): 93.466018
Using microphone 2 as closest mic - (0 left, 1 right)
Solver: r = 270.143144, theta = -50.299987
ID: 12, 267.492191,266.533982,-410.625986,-50.299987
Done.
Angles: [96.51421132334087, 92.10374588045173, 111.6226058391794]
Errors: [1.0903235479269324, 1.1805160391749407, 2.7137678054904253]

Angle (CW): 96.558637
Using microphone 2 as closest mic - (0 left, 1 right)
Solver: r = 0.257210, theta = 299.999985
Angles: [96.42382144379854, 92.07114055821768, 111.50449425151291]
Errors: [1.0910728305627395, 1.1800949022622775, 2.727960616719987]

Angle (CW): 96.472427
Using microphone 2 as closest mic - (0 left, 1 right)
Solver: r = 0.256806, theta = 299.999985
ID: 13, 263.441363,263.527573,299.999985,299.999985
Done.


locate.py:363: RuntimeWarning: invalid value encountered in divide
  CORR = CORR / (np.abs(SIG1) * np.abs(np.conj(SIG2)))


Angles: [108.55055895646997, 149.453190305103, 90.546809694897]
Errors: [1.0203037909921615, 104.78352855705414, 104.78352855705414]

Angle (CW): 113.878740
Using microphone 2 as closest mic - (0 left, 1 right)
Solver: r = 108704.224767, theta = 297.108522
Angles: [107.6754251545761, 81.76275605093957, 133.95368520544324]
Errors: [1.023589528312158, 1.076742645441642, 1.4407630245727059]

Angle (CW): 104.908149
Using microphone 2 as closest mic - (0 left, 1 right)
Solver: r = 2071.990631, theta = 314.343336
ID: 14, 246.121260,255.091851,297.108522,314.343336
Done.
Angles: [122.1473193309101, 117.92172006807483, 125.93847922774137]
Errors: [1.0007027037093474, 1.882964674529475, 1.7038206346580742]

Angle (CW): 122.197198
Using microphone 2 as closest mic - (0 left, 1 right)
Solver: r = 3.618107, theta = 299.999983
Angles: [120.70641529859634, 117.25904336093386, 124.40379494622324]
Errors: [1.0000760102987931, 1.8489697940164507, 1.7698436771047994]

Angle (CW): 120.844899
Using microp

In [8]:
# Assuming comment is the _actual_ angle
for i in range(len(ang_solver_raw)):
    print '%f,%f,%f,%f,%s' % (ang_far_raw[i], ang_far_butter[i], ang_solver_raw[i], ang_solver_butter[i], experiments[i][1])

359.355950,359.489288,419.999996,422.600362,0
359.752469,359.870150,419.999996,426.486911,0
5.422860,4.666675,59.999997,59.999997,0
343.313761,343.905189,419.999996,419.999996,30
343.046074,343.223037,419.999996,422.926866,30
343.273591,343.456280,422.715977,423.193557,30
294.783929,294.282074,299.999983,299.999983,60
294.832244,294.739550,299.999983,299.999983,60
295.310932,295.267688,299.999983,299.999983,60
0.000000,0.000000,0.000000,0.000000,90
261.928582,262.285534,299.999985,299.999985,90
267.492191,266.533982,-410.625986,-50.299987,90
263.441363,263.527573,299.999985,299.999985,90
246.121260,255.091851,297.108522,314.343336,120
237.802802,239.155101,299.999983,299.999983,120
240.541113,234.011503,299.999985,299.999985,120
220.792642,221.662993,299.810273,299.999983,150
220.294649,220.993107,299.999983,299.999983,150
221.907551,222.003134,299.999983,299.999983,150
221.889432,222.485268,299.999983,299.999983,150
222.133433,222.520433,299.999983,299.999984,150
222.192362,222.657145